# **We begin by importing the necessary libraries and loading the anime dataset into a DataFrame to preview the first few records using df.head().** 

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import  matplotlib.pyplot as plt
import seaborn as sns
#from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import train_test_split
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.metrics import confusion_matrix, classification_report

df = pd.read_csv('/kaggle/input/myanimelist-dataset/final_animedataset.csv')

df.head()


,username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,rank,popularity,genre
0,karthiga,21,9,2255153,Female,One Piece,TV,Manga,8.54,423868,91.0,35,"Action, Adventure, Comedy, Super Power, Drama,..."
1,karthiga,59,7,2255153,Female,Chobits,TV,Manga,7.53,175388,1546.0,188,"Sci-Fi, Comedy, Drama, Romance, Ecchi, Seinen"
2,karthiga,74,7,2255153,Female,Gakuen Alice,TV,Manga,7.77,33244,941.0,1291,"Comedy, School, Shoujo, Super Power"
3,karthiga,120,7,2255153,Female,Fruits Basket,TV,Manga,7.77,167968,939.0,222,"Slice of Life, Comedy, Drama, Romance, Fantasy..."
4,karthiga,178,7,2255153,Female,Ultra Maniac,TV,Manga,7.26,9663,2594.0,2490,"Magic, Comedy, Romance, School, Shoujo"


In [1]:
df.columns

NameError: name 'df' is not defined

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35305695 entries, 0 to 35305694
Data columns (total 13 columns):
 #   Column      Dtype  
---  ------      -----  
 0   username    object 
 1   anime_id    int64  
 2   my_score    int64  
 3   user_id     int64  
 4   gender      object 
 5   title       object 
 6   type        object 
 7   source      object 
 8   score       float64
 9   scored_by   int64  
 10  rank        float64
 11  popularity  int64  
 12  genre       object 
dtypes: float64(2), int64(5), object(6)
memory usage: 3.4+ GB


**These values could be adjusted if working locally since Kaggle and Colab can not run these lines of code....**

In [3]:
df_small = df.sample(n=1000, random_state=42).reset_index(drop=True)


 **To prepare the data for vectorization, we create a new tags column by combining key textual features like title, genre, type, and source into a single string for each anime entry.**

In [4]:
df_small['tags'] = (
    df['title'].fillna('') + ' ' +
    df['genre'].fillna('').str.replace(',', ' ') + ' ' +
    df['type'].fillna('') + ' ' +
    df['source'].fillna('')
)

In [6]:
df_small.head()

,username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,rank,popularity,genre,tags
0,Nioko13,11531,0,74301,Female,UN-GO: Inga-ron,Movie,Novel,7.70,14566,1121.0,2229,"Mystery, Supernatural",One Piece Action Adventure Comedy Super Pow...
1,Dust_,34599,0,5595739,Male,Made in Abyss,TV,Web manga,8.91,166905,20.0,178,"Sci-Fi, Adventure, Mystery, Drama, Fantasy",Chobits Sci-Fi Comedy Drama Romance Ecchi ...
2,Nishimiya_Sensei,8937,0,5808728,Male,Toaru Majutsu no Index II,TV,Light novel,7.79,130053,912.0,294,"Action, Sci-Fi, Super Power, Magic",Gakuen Alice Comedy School Shoujo Super Pow...
3,ddbad,317,5,2503685,Male,Final Fantasy VII: Advent Children,Movie,Game,7.88,112969,733.0,500,"Action, Super Power, Fantasy",Fruits Basket Slice of Life Comedy Drama Ro...
4,natsumi-chan,4246,10,109945,Female,Eureka Seven: Pocket ga Niji de Ippai,Movie,Original,7.22,19189,2715.0,1867,"Adventure, Mecha, Romance, Sci-Fi",Ultra Maniac Magic Comedy Romance School S...


 **We drop the title, genre, type and source columns from the main dataset to reduce memory usage during processing, but store it separately in a metadata DataFrame for future reference or display.**

In [7]:
metadata = df[['title', 'genre', 'type', 'source']]
df_small.drop(['title', 'genre', 'type', 'source'], axis=1, inplace=True)


In [8]:
df_small.head()

,username,anime_id,my_score,user_id,gender,score,scored_by,rank,popularity,tags
0,Nioko13,11531,0,74301,Female,7.70,14566,1121.0,2229,One Piece Action Adventure Comedy Super Pow...
1,Dust_,34599,0,5595739,Male,8.91,166905,20.0,178,Chobits Sci-Fi Comedy Drama Romance Ecchi ...
2,Nishimiya_Sensei,8937,0,5808728,Male,7.79,130053,912.0,294,Gakuen Alice Comedy School Shoujo Super Pow...
3,ddbad,317,5,2503685,Male,7.88,112969,733.0,500,Fruits Basket Slice of Life Comedy Drama Ro...
4,natsumi-chan,4246,10,109945,Female,7.22,19189,2715.0,1867,Ultra Maniac Magic Comedy Romance School S...


In [9]:
from sklearn.feature_extraction.text import CountVectorizer


In [10]:
cv = CountVectorizer(max_features=5000, stop_words='english')



In [11]:
cv

CountVectorizer(max_features=5000, stop_words='english')

# **The Hardest Part** 

In [13]:
vectorized = cv.fit_transform(df_small['tags'].values.astype('U')).toarray()


In [14]:
vectorized

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [22]:
vectorized.shape

(1000, 1406)

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
sim = cosine_similarity(vectorized)

In [24]:

dist = sorted(list(enumerate(sim[0])), reverse=True, key=lambda x: x[1])

In [25]:
dist

[(0, 1.0),
 (53, 1.0),
 (754, 1.0),
 (9, 0.7627700713964739),
 (55, 0.7627700713964739),
 (757, 0.7627700713964739),
 (987, 0.7526178090063818),
 (84, 0.7272727272727273),
 (228, 0.7272727272727273),
 (243, 0.7272727272727273),
 (766, 0.7272727272727273),
 (814, 0.7272727272727273),
 (819, 0.7272727272727273),
 (929, 0.7272727272727273),
 (945, 0.7006490497453707),
 (966, 0.7006490497453707),
 (19, 0.6963106238227914),
 (60, 0.6963106238227914),
 (104, 0.6963106238227914),
 (546, 0.6963106238227914),
 (639, 0.6963106238227914),
 (775, 0.6963106238227914),
 (881, 0.6963106238227914),
 (899, 0.6963106238227914),
 (964, 0.6784005252999683),
 (965, 0.6784005252999683),
 (333, 0.6689936080056726),
 (651, 0.6689936080056726),
 (844, 0.6689936080056726),
 (106, 0.6674238124719147),
 (237, 0.6674238124719147),
 (776, 0.6674238124719147),
 (817, 0.6674238124719147),
 (301, 0.6674238124719146),
 (365, 0.6674238124719146),
 (618, 0.6674238124719146),
 (835, 0.6674238124719146),
 (852, 0.667423812

**Created recommend function and tested it.**

In [29]:
def recommend(index, sim, metadata, top_n=5):
    dist = sorted(
        list(enumerate(sim[index])), 
        reverse=True, 
        key=lambda x: x[1]
    )

    seen_titles = set()
    recommendations = []

    for idx, score in dist:
        title = metadata.iloc[idx]['title']
        if title not in seen_titles and idx != index:
            seen_titles.add(title)
            recommendations.append((idx, score))
        if len(recommendations) == top_n:
            break

    for idx, score in recommendations:
        print(f"Title: {metadata.iloc[idx]['title']}")
        print(f"Genre: {metadata.iloc[idx]['genre']}")
        print(f"Type: {metadata.iloc[idx]['type']}")
        print(f"Source: {metadata.iloc[idx]['source']}")
        print(f"Similarity Score: {score:.4f}")
        print("---")


In [32]:
recommend(50, sim, metadata)


Title: Tonari no Kaibutsu-kun
Genre: Slice of Life, Comedy, Romance, School, Shoujo
Type: TV
Source: Manga
Similarity Score: 1.0000
---
Title: Sora no Manimani
Genre: Comedy, Romance, School, Slice of Life
Type: TV
Source: Manga
Similarity Score: 0.7035
---
Title: Kareshi Kanojo no Jijou
Genre: Comedy, Drama, Romance, School, Shoujo, Slice of Life
Type: TV
Source: Manga
Similarity Score: 0.6963
---
Title: Ao Haru Ride
Genre: Comedy, Drama, Romance, School, Shoujo, Slice of Life
Type: TV
Source: Manga
Similarity Score: 0.6963
---
Title: Special A
Genre: Comedy, Romance, School, Shoujo
Type: TV
Source: Manga
Similarity Score: 0.6838
---
